# Workflow Integration

> End-to-end workflow: Finnish texts → Translation → Audio → Anki cards

In [ ]:
#| default_exp workflow

In [ ]:
#| export
from pathlib import Path
from suomi.tsv import *
from suomi.mp3 import *
from suomi.anki import *

## Main Workflow Function

In [ ]:
#| export
def create_cards(
    texts: list[str],
    deck: str,
    tags: str | list[str] = "lang/fi",
    output_dir: str = "output"
) -> None:
    """
    Create Anki cards from Finnish texts (end-to-end workflow).
    
    Steps:
    1. Translate Finnish → English/Japanese (OpenAI API)
    2. Generate audio files (Piper TTS)
    3. Update TSV with file paths
    4. Upload to Anki (AnkiConnect)
    
    Args:
        texts: List of Finnish words/phrases to create cards for
        deck: Anki deck name (e.g., "06::Daily")
        tags: Tags for the cards (string or list), hierarchical format supported
              Examples: "src/daily", "src/class,level/A1", ["src/medical", "urgent"]
              Note: "lang/fi" is always auto-included
        output_dir: Directory for TSV and audio files (default: "output")
    
    Example:
        >>> create_cards(
        ...     texts=["kissa", "koira"],
        ...     deck="06::Daily",
        ...     tags="src/daily"
        ... )
    """
    # Create safe filename from deck name
    tsv_name = deck.replace("::", "_") + ".tsv"
    tsv_path = f"{output_dir}/{tsv_name}"
    audio_dir = f"{output_dir}/audio"
    images_dir = f"{output_dir}/images"
    
    # Ensure output directory exists
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # 1. Translate and create TSV
    texts2tsv(texts, tsv_path, tags=tags)
    
    # 2. Generate audio files
    mp3s(tsv_path, output_dir=audio_dir)
    
    # 3. Update TSV with file paths (search both audio and images directories)
    update_tsv_media_paths(tsv_path, dirs=[audio_dir, images_dir])
    
    # 4. Upload to Anki
    addnotes(deck, tsv_path)

## Example Usage

In [ ]:
#| eval: false
# Daily vocabulary
create_cards(
    texts=["kissa", "koira"],
    deck="06::Daily",
    tags="src/daily"
)

In [ ]:
#| eval: false
# Class sentences with multiple tags
create_cards(
    texts=["Minulla on päänsärky", "Kiitos paljon"],
    deck="06::Class",
    tags="src/class,level/A1"
)

In [ ]:
#| eval: false
# Medical vocabulary with list of tags
create_cards(
    texts=["lääkäri", "sairaala", "resepti"],
    deck="06::Medical",
    tags=["src/medical", "urgent", "topic/health"]
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()